In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

'''Plotly visualization '''
import plotly.express as px
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
py.init_notebook_mode(connected = True) # Required to use plotly offline in jupyter notebook

'''Display markdown formatted output like bold, italic bold etc.'''
from IPython.display import Markdown
def bold(string):
    display(Markdown(string))

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# data clening
df = pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
data = df.copy()
data['Date']=data['Date'].apply(pd.to_datetime)
data.drop(['Sno','Time'],axis=1,inplace=True)

#colecting present data
from datetime import date
data_apr = data[data['Date'] >= pd.Timestamp(date(2020,4,19))]

#prepare data satewise
state_cases = data_apr.groupby('State/UnionTerritory')['Confirmed','Deaths','Cured'].max().reset_index()
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Deaths']+state_cases['Cured'])
state_cases['Death_Rate(per_100)'] = np.round(100*state_cases['Deaths']/state_cases['Confirmed'],2)

In [ ]:
bold("** Present Condition in India: Confirmed cases, Deaths, Recovered of 2019-nCoV**")
print('Total Confirmed:',state_cases['Confirmed'].sum())
print('Total Cured:',state_cases['Cured'].sum())
print('Total Deaths:',state_cases['Deaths'].sum())
print('Death Rate (per 100):',np.round(100*state_cases['Deaths'].sum()/state_cases['Confirmed'].sum(),2))

In [ ]:
bold("** STATE WISE CONFIRMED, DEATH AND CURED CASES of 2019-nCoV**")
state_cases.sort_values('Confirmed', ascending=False).style.background_gradient(cmap='Blues',subset=['Confirmed'])\
.background_gradient(cmap='Reds',subset=['Deaths'])\
.background_gradient(cmap='Greens',subset=['Cured'])\
.background_gradient(cmap='pink',subset=['Active'])\
.background_gradient(cmap='Purples',subset=["Death_Rate(per_100)"])

In [ ]:
fig = px.bar (state_cases.sort_values('Confirmed',ascending=False).sort_values('Confirmed',ascending=True),
             y="Confirmed",x="State/UnionTerritory",
             title='Total confirmed cases',
             text= 'Confirmed',
             orientation='v',
             width=1000 ,height=1000
             )
fig.update_traces(marker_color='#FFA500', opacity=0.8, textposition='outside')

fig.update_layout(template = 'plotly_white')
fig.show()

In [ ]:
fig = px.bar (state_cases.sort_values('Cured',ascending=False).sort_values('Cured',ascending=True),
             y="Cured",x="State/UnionTerritory",
             title='Total Cured cases',
             text= 'Cured',
             orientation='v',
             width=1000 ,height=700,
             )
fig.update_traces(marker_color='green', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()

In [ ]:
fig = px.bar (state_cases.sort_values('Deaths',ascending=False).sort_values('Deaths',ascending=True),
             y="Deaths",x="State/UnionTerritory",
             title='Total Deaths cases',
             text= 'Deaths',
             orientation='v',
             width=1000 ,height=700,
             )
fig.update_traces(marker_color='Red', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()

In [ ]:
fig = px.bar (state_cases.sort_values('Active',ascending=False).sort_values('Active',ascending=True),
             y="Active",x="State/UnionTerritory",
             title='Total Active cases',
             text= 'Active',
             orientation='v',
             width=1000 ,height=700,
             )
fig.update_traces(marker_color='purple', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()

In [ ]:
fig = px.bar (state_cases.sort_values('Death_Rate(per_100)',ascending=False).sort_values('Death_Rate(per_100)',ascending=True),
             y="Death_Rate(per_100)",x="State/UnionTerritory",
             title='Death_Rate(per_100)',
             text= 'Death_Rate(per_100)',
             orientation='v',
             width=1000 ,height=700,
             )
fig.update_traces(marker_color='pink', opacity=0.8, textposition='inside')

fig.update_layout(template = 'plotly_white')
fig.show()

In [ ]:

spread = pd.read_csv('/kaggle/input/covid19-in-india/covid_19_india.csv')
spread['Date'] = pd.to_datetime(spread['Date'], dayfirst=True,)
# spread['Date'] = spread['Date'].apply(pd.to_datetime)

# from datetime import date
# spread = spread[spread['Date'] > pd.Timestamp(date(2020,4,20))]
# [spread['State/UnionTerritory']=='Tamil Nadu'].reset_index(drop=True)
india_sprea = spread
india_sprea['Active'] = india_sprea['Confirmed'] - (india_sprea['Deaths'] + india_sprea['Cured'])
india_spread = india_sprea.groupby('Date')['Confirmed','Deaths','Cured', 'Active'].sum().reset_index()

india_spread['Confirmed_diff'] = india_spread['Confirmed'].diff()
india_spread['Death_diff'] = india_spread['Deaths'].diff()
india_spread['Recovered_diff'] = india_spread['Cured'].diff()
# date_india_spread = india_spread.groupby('Date')['Confirmed','Deaths','Cured', 'Active'].sum().reset_index()

In [ ]:
#ploting spread over time
date_india_spread = india_sprea.groupby('Date')['Confirmed','Deaths','Cured', 'Active'].sum().reset_index()

trace1 = go.Scatter(
                x=date_india_spread['Date'],
                y=date_india_spread['Confirmed'],
                name="Confirmed",
                mode='lines+markers',
                line_color='orange')
trace2 = go.Scatter(
                x=date_india_spread['Date'],
                y=date_india_spread['Deaths'],
                name="Deaths",
                mode='lines+markers',
                line_color='red')

trace3 = go.Scatter(
                x=date_india_spread['Date'],
                y=date_india_spread['Cured'],
                name="Cured",
                mode='lines+markers',
                line_color='green')
trace4 = go.Scatter(
                x=date_india_spread['Date'],
                y=date_india_spread['Active'],
                name="Active",
                mode='lines+markers',
                line_color='blue')

layout = go.Layout(template="ggplot2", width=700, height=500, title_text = '<b>Spread of the Coronavirus In India Over Time </b>',
                  font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig = go.Figure(data = [trace1,trace2,trace3, trace4], layout = layout)
fig.show()

# plot daily cases
colors = ['#FFA500']*80
colors[-1] = 'crimson'
fig = px.bar(india_spread, 
             x="Date", y="Confirmed_diff", 
             title='<b>New Confirm Cases Per Day In India</b>', 
             orientation='v', 
             width=700, height=600)
fig.update_traces(marker_color=colors, opacity=0.8)

fig.update_layout(template = 'plotly_white',font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig.show()
# plot daily death cases
colors = ['#FFA500']*80
colors[-1] = 'crimson'
fig = px.bar(india_spread, 
             x="Date", y="Death_diff", 
             title='<b>New Death Cases Per Day In India</b>', 
             orientation='v', 
             width=700, height=600)
fig.update_traces(marker_color=colors, opacity=0.8)

fig.update_layout(template = 'plotly_white',font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig.show()
# plot daily Recovered cases
colors = ['#FFA500']*80
colors[-1] = 'crimson'
fig = px.bar(india_spread, 
             x="Date", y="Recovered_diff", 
             title='<b>New Recovered Cases Per Day In India</b>', 
             orientation='v', 
             width=700, height=600)
fig.update_traces(marker_color=colors, opacity=0.8)

fig.update_layout(template = 'plotly_white',font=dict(family="Arial, Balto, Courier New, Droid Sans",color='black'))
fig.show()
